In [1]:
import requests

In [3]:
url = 'https://2gis.ru/novosibirsk/search/{}%2C%20{}'

In [4]:
import pandas as pd

In [6]:
data = pd.read_csv('./data/train/rosstat_population_all_cities.csv')

In [7]:
data.head()

,geo_h3_10,city,population,lat,lon
0,8a0bb0200907fff,Новосибирск,12,54.942110,83.146356
1,8a0bb0200927fff,Новосибирск,8,54.942144,83.144157
2,8a0bb020092ffff,Новосибирск,28,54.943220,83.145270
3,8a0bb0200937fff,Новосибирск,12,54.941034,83.145243
4,8a0bb0200977fff,Новосибирск,10,54.944296,83.146384


In [34]:
coords = data.iloc[:, [-2, -1]]

In [35]:
coords

,lat,lon
0,54.942110,83.146356
1,54.942144,83.144157
2,54.943220,83.145270
3,54.941034,83.145243
4,54.944296,83.146384
...,...,...
34850,59.999184,30.137007
34851,59.999851,30.133155
34852,60.000634,30.134712
34853,59.998735,30.133524


In [18]:
import numpy as np

In [36]:
coords = coords.to_numpy()

In [37]:
x, y = coords[0]

In [38]:
x, y

(54.942109503874754, 83.14635611152772)

In [42]:
url.format(x, y)

'https://2gis.ru/novosibirsk/search/54.942109503874754%2C%2083.14635611152772'

In [56]:
s = requests.Session()

In [53]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

In [60]:
r = s.get(url.format(x, y), headers=headers)

In [61]:
r.text

'<!DOCTYPE HTML>\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<meta name="viewport" content="width=device-width,initial-scale=1.0"/>\n<style>\n.error {\n    color: #ff0000;\n}\n.ack {\n    color: #0000ff;\n}\n.button7:hover {\n    -webkit-filter: brightness(0.9);\n    filter: brightness(0.9);\n}\n.button7:active {\n    -webkit-filter: brightness(0.7);\n    filter: brightness(0.7);\n}\n.button7 {\n-webkit-appearance: none;\n    font-size: 13px;\n    line-height: 16px;\n    font-weight: 400;\n    font-variant-numeric: oldstyle-nums;\n    position: relative;\n    z-index: 0;\n    box-sizing: border-box;\n    display: inline-block;\n    cursor: pointer;\n    color: rgb(255, 255, 255);\n    border-width: initial;\n    border-style: none;\n    border-color: initial;\n    border-image: initial;\n    margin: 0px;\n    padding: 12px 24px;\n    text-decoration: none;\n    border-radius: 4px;\n    overflow: hidden;\n    background: rgb(2, 142, 255);\n    

In [62]:
url_yandex = 'https://yandex.ru/maps/65/novosibirsk/?ll={0}%2C{1}&mode=search&sll={0}%2C{1}&text={0}%2C{1}&z=17.43'

In [88]:
url_yandex.format(x, y)

'https://yandex.ru/maps/65/novosibirsk/?ll=54.94110375628233%2C83.14084413076488&mode=search&sll=54.94110375628233%2C83.14084413076488&text=54.94110375628233%2C83.14084413076488&z=17.43'

In [ ]:
r = s.get(url_yandex.format(x, y), headers=headers)
r.text

In [69]:
with open('./index.html', 'w') as f:
    f.write(r.text)

In [70]:
s = r.text

In [77]:
s.split('class="breadcrumbs-view__breadcrumb _outline"')[4].split('aria-label')[1].split('>')[1].split('<')[0]

'Первомайский район'

In [79]:
rez = dict()

In [81]:
session = requests.Session()

In [143]:
j = 1
for x, y in coords[13059:15000]:
    r = session.get(url_yandex.format(x, y), headers=headers)
    s = r.text
    if ' район' in s.lower():
        i = s.find(' район')
        rez[(x, y)] = s[i-20:i+6]
    elif 'микрорайон' in s.lower():
        i = s.find('микрорайон')
        rez[(x, y)] = s[i-20:i+11]
    j += 1
    print(f'{j}/{5000}\r', end='')

KeyboardInterrupt: 

In [144]:
len(rez)

7285

In [90]:
r = session.get('https://yandex.ru/maps/65/novosibirsk/?ll=54.94110375628233%2C83.14084413076488&mode=search&sll=54.94110375628233%2C83.14084413076488&text=54.94110375628233%2C83.14084413076488&z=17.43', headers=headers)

In [91]:
text = r.text

In [94]:
i = text.find('район')
text[i-20:i+5]

'ntent="Первомайский район'

In [136]:
5918 + 7141

13059

In [ ]:
import csv
csv_cols = ['lat', 'lon', 'reg']
dict_data = [dict(zip([csv_cols], [x, y, rez[(x, y)]])) for x, y in rez.keys()]
with open('saved_data_regs.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)